# Notebook SAE 202

## Partie II


In [ ]:
import sounddevice as sd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sc
import soundfile as sf
import time
import threading as th


: 

### Reception

In [6]:
print(sd.query_devices())

nombre_blocs = 0

def print_number(indata, frames, time, status):
    global nombre_blocs
    print(f"Réception du chunk numéro {nombre_blocs} dont le premier éléments vaut {indata[0]}")
    nombre_blocs += 1
stream = sd.InputStream(callback=print_number, blocksize=10000, channels=1)
print("Démarrage du stream")
stream.start()
time.sleep(30)
stream.stop()
print("FIn du stream")
print(f"{nombre_blocs} bloc ont été reçu")


   0 Mappeur de sons Microsoft - Input, MME (2 in, 0 out)
>  1 Microphone (USB Audio Device), MME (1 in, 0 out)
   2 Microphone (Conexant ISST Audio, MME (2 in, 0 out)
   3 Mappeur de sons Microsoft - Output, MME (0 in, 2 out)
<  4 Haut-parleurs (USB Audio Device, MME (0 in, 2 out)
   5 Haut-parleur (Conexant ISST Aud, MME (0 in, 6 out)
   6 Réseau de microphones (Conexant ISST Audio capture), Windows WDM-KS (2 in, 0 out)
   7 Mixage stéréo (Conexant ISST Audio capture), Windows WDM-KS (2 in, 0 out)
   8 Output 1 (Conexant ISST Audio output), Windows WDM-KS (0 in, 2 out)
   9 Output 2 (Conexant ISST Audio output), Windows WDM-KS (0 in, 6 out)
  10 Input (Conexant ISST Audio output), Windows WDM-KS (2 in, 0 out)
  11 Headphones (Conexant ISST Audio headphone), Windows WDM-KS (0 in, 2 out)
  12 Casque (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Redmi Buds 4)), Windows WDM-KS (0 in, 1 out)
  13 Casque (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Redmi Buds 4)), Windows 

### Emission

In [1]:
print(sd.query_devices())

   0 Mappeur de sons Microsoft - Input, MME (2 in, 0 out)
>  1 Microphone (USB Audio Device), MME (2 in, 0 out)
   2 AI Noise-cancelling Input (ASUS, MME (2 in, 0 out)
   3 Réseau de microphones (Realtek(, MME (2 in, 0 out)
   4 Headset Microphone (Oculus Virt, MME (2 in, 0 out)
   5 Mappeur de sons Microsoft - Output, MME (0 in, 2 out)
<  6 Haut-parleurs (USB Audio Device, MME (0 in, 2 out)
   7 AI Noise-cancelling Output (ASU, MME (0 in, 2 out)
   8 Haut-parleurs (Realtek(R) Audio, MME (0 in, 2 out)
   9 Casque (Oculus Virtual Audio De, MME (0 in, 2 out)
  10 Pilote de capture audio principal, Windows DirectSound (2 in, 0 out)
  11 Microphone (USB Audio Device), Windows DirectSound (2 in, 0 out)
  12 AI Noise-cancelling Input (ASUS Utility), Windows DirectSound (2 in, 0 out)
  13 Réseau de microphones (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
  14 Headset Microphone (Oculus Virtual Audio Device), Windows DirectSound (2 in, 0 out)
  15 Périphérique audio principal, Windows

In [8]:
fs = 10000

def callback(indata, outdata, frames, time, status):
    if status:
        print(status)

    outdata[:] = indata
stream = sd.Stream(samplerate=fs, callback=callback)

try:
    with stream:
        print('#' * 80)
        print('press Return to quit')
        print('#' * 80)
        input()

except KeyboardInterrupt:
    exit('')


################################################################################
press Return to quit
################################################################################


In [17]:
event = th.Event()

try:
    data, fs = sf.read('NR4.wav', always_2d=True)
    current_frame = 0

    def callback(outdata, frames, time, status):
        global current_frame
        if status:
            print(status)

        chunksize = min(len(data) - current_frame, frames )
        audio = data[current_frame:current_frame + chunksize]
        outdata[:] = audio
        if chunksize < frames:
            outdata[chunksize] = 0
            raise sd.CallbackStop()
        current_frame += chunksize

    stream = sd.OutputStream(samplerate = fs, callback=callback, finished_callback=event.set)

    with stream :
        event.wait()

except KeyboardInterrupt:
    exit('\nInterrupted by  user')
    


Exception ignored from cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x000001F189E2BBA0>:
Traceback (most recent call last):
  File "C:\Users\enzoj\anaconda3\envs\rosa\Lib\site-packages\sounddevice.py", line 862, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\enzoj\anaconda3\envs\rosa\Lib\site-packages\sounddevice.py", line 2687, in _wrap_callback
    callback(*args)
  File "C:\Users\enzoj\AppData\Local\Temp\ipykernel_9672\2035060037.py", line 14, in callback
ValueError: could not broadcast input array from shape (2048,1) into shape (2576,2)
